# **Extração das imagens do .rec**

In [2]:
import os
import numpy as np
np.bool = np.bool_
import mxnet as mx
import cv2
import pickle
from tqdm.notebook import tqdm
import pandas as pd

In [5]:
rec_path = '../data/CASIA/faces_webface_112x112/'
output_dir = '../data/CASIA/'

In [25]:
import os
import pandas as pd
import cv2
import mxnet as mx
from tqdm import tqdm

def load_mx_rec(df, rec_path, save_path, write_img=True):
    if write_img:
        if not os.path.isdir(save_path + "/casia-faces"):
            os.makedirs(save_path + "/casia-faces")

    imgrec = mx.recordio.MXIndexedRecordIO(
        os.path.join(rec_path, 'train.idx'),
        os.path.join(rec_path, 'train.rec'), 'r')
    img_info = imgrec.read_idx(0)
    header, _ = mx.recordio.unpack(img_info)
    max_idx = int(header.label[0])

    file_path = os.path.join(save_path, "casia-faces")

    if not os.path.isdir(file_path):
        os.makedirs(file_path)

    data_list = []  # Lista para armazenar dados antes de escrever no CSV

    for idx in tqdm(range(1, max_idx), desc="Extracting images"):
        img_info = imgrec.read_idx(idx)
        header, img = mx.recordio.unpack_img(img_info)
        label = int(header.label)
        img_path = f"{label}_{idx}.jpg"

        if write_img and img_path in df['path'].values:
            cv2.imwrite(os.path.join(file_path, img_path), img)

        data_list.append([img_path, label])

    # Criar DataFrame e salvar em CSV
    new_df = pd.DataFrame(data_list, columns=['path', 'id'])
    new_df.to_csv(os.path.join(save_path, "casia.csv"), index=False)

In [26]:
load_mx_rec(None, rec_path, output_dir, write_img=False) # Não escrever imagens, apenas obter CSV

Extracting images: 100%|██████████| 490623/490623 [00:41<00:00, 11778.79it/s]


# **Limpeza**

In [13]:
# Carregar DataFrame
df = pd.read_csv(os.path.join(output_dir, 'casia.csv'))

# Filtros e seleção de amostras
df_clean = df.groupby('id').filter(lambda x: len(x) >= 100)
df_clean = df_clean.groupby('id').filter(lambda x: len(x) <= 300)

print(f'Total de imagens no df: {df_clean.shape[0]:,}')
print(f"Total de identidades no df: {df_clean['id'].nunique()}")

Total de imagens no df: 124,134
Total de identidades no df: 784


In [14]:
df_clean['id'].value_counts().min(), df_clean['id'].value_counts().max(), df_clean['id'].value_counts().mean()

(100, 300, 158.3341836734694)

In [29]:
load_mx_rec(df_clean, rec_path, output_dir, write_img=True) # Escreve imagens, somente com amostras selecionadas

Extracting images: 100%|██████████| 490623/490623 [11:16<00:00, 725.26it/s]


In [30]:
qtd = len(os.listdir(os.path.join(output_dir, 'casia-faces')))
print(f'Total de imagens na pasta: {qtd:,}')

Total de imagens na pasta: 124,134


# **Separar em treino e teste**

In [15]:
val_ids = df_clean['id'].value_counts().nsmallest(5).index
val_df = df_clean[df_clean['id'].isin(val_ids)].copy()

remaining_ids = df_clean[~df_clean['id'].isin(val_ids)]['id'].unique()
test_df = df_clean[df_clean['id'].isin(remaining_ids)].copy()
test_df = test_df.groupby('id').head(10)

train_df = df_clean[~df_clean.index.isin(test_df.index) & ~df_clean.index.isin(val_df.index)].copy()

In [16]:
# Renomear coluna id para id_original
train_df.rename(columns={'id': 'id_original'}, inplace=True)
val_df.rename(columns={'id': 'id_original'}, inplace=True)
test_df.rename(columns={'id': 'id_original'}, inplace=True)

# Resetar os índices
train_df.loc[:, 'id'], _ = pd.factorize(train_df['id_original'])
test_df.loc[:, 'id'], _ = pd.factorize(test_df['id_original'])
val_df.loc[:, 'id'], _ = pd.factorize(val_df['id_original'])

In [17]:
print(f"train_df: {train_df.shape[0]:,} imagens | {train_df['id'].nunique()} identidades | média de {train_df['id'].value_counts().mean():.2f} imagens por identidade")
print(f"test_df: {test_df.shape[0]:,} imagens | {test_df['id'].nunique()} identidades | média de {test_df['id'].value_counts().mean():.2f} imagens por identidade")
print(f"val_df: {val_df.shape[0]:,} imagens | {val_df['id'].nunique()} identidades | média de {val_df['id'].value_counts().mean():.2f} imagens por identidade")

train_df: 115,844 imagens | 779 identidades | média de 148.71 imagens por identidade
test_df: 7,790 imagens | 779 identidades | média de 10.00 imagens por identidade
val_df: 500 imagens | 5 identidades | média de 100.00 imagens por identidade


In [18]:
train_df.to_csv(os.path.join(output_dir, 'train.csv'), index=False)
test_df.to_csv(os.path.join(output_dir, 'test.csv'), index=False)
val_df.to_csv(os.path.join(output_dir, 'val.csv'), index=False)

In [22]:
# Verificar a mesma pessoa em diferentes datasets (treino e teste)
erro = False
for i in range(train_df['id'].nunique()):
    if train_df[train_df['id'] == i].iloc[0]['id_original'] != test_df[test_df['id'] == i].iloc[0]['id_original']:
        print(f"Erro: {train_df[train_df['id'] == i].iloc[0]['id_original']} != {test_df[test_df['id'] == i].iloc[0]['id_original']}")
        erro = True

if not erro:
    print("Nenhum erro encontrado")

Nenhum erro encontrado
